In [ ]:
import random
import os
import numpy as np
from sklearn import svm
from sklearn.model_selection import cross_val_predict, cross_val_score,KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from fastdtw import fastdtw

In [ ]:
MY_NODE_NO =25


MAX_FRAME =164

NO_FEATURES=1629


NO_CLASSES =60

temp_path='DTW/templates.txt'
dest_path='DTW/'
crossValidationDataPaths=['NORMALIZED/NumericalData/ML/Training/RightHand','NORMALIZED/NumericalData/ML/Training/LeftHand','NORMALIZED/NumericalData/ML/Test/RightHand','NORMALIZED/NumericalData/ML/Test/LeftHand']
templatePaths=['NORMALIZED/NumericalData/ML/Test/RightHand']

In [ ]:


def distance (x1,x2):
    return np.linalg.norm(x1-x2)

In [ ]:
def my_flatten(xss):
    return[x for xs in xss for x in xs]

In [ ]:
print('THIS CODE IN THIS CELL SHOULD NOT EXECUTED AS DTW DISTANCE CALCULATION IS PROHIBITLY EXPENSIVE')
print ('RATHER DOWNLOAD THE PRECALCULATED DTW DISTANCE DATASET FROM KAGGLE AND COPY TO EMPTY DTW FOLDER AVAILABLE (OVERWRITE) IN CURRENT DIRECTORY')
print ('KAGGLE DATASET NAME: DTW_BdSLW60')

print ('')
print ('')
print ('Used 60 computers to calculate the distances in more than 5 days')
NO_TEMPLATE =1
template_paths=[]
for path in templatePaths:
    classFolders= os.listdir(path)
    for classFolder in classFolders:
        classFolder_path=f'{path}/{classFolder}'
        classTrials =os.listdir(classFolder_path)
        tc=0;
        for trial in classTrials:
            
            if(tc <NO_TEMPLATE):
               trialPath =f'{classFolder_path}/{trial}'
               template_paths.append(trialPath)           
            
            tc=tc+1

# read the template paths from file

if(os.path.exists(temp_path)):
    #read templates from the file
    template_paths=[]
    file=open(temp_path,"r")
    lines=file.readlines()
    count=0
    for line in lines:
        count+=1
        if(len(line)>0):
            template_paths.append(line.strip('\n\r'))
    file.close()
else:
    #write the lines in the file 
    file=open(temp_path,"w")
    for path in template_paths:
        file.writeLine(path)
    file.close()



templatesDTW ={}
for template_path in template_paths:
    label = template_path.split('/')
    label = label[len(label)-1].split('_')[0]
      
    npy =np.load(template_path)
    #npy_matrix=npy.reshape(-1,1629)
    #now just one templat per class possible
    
    try:
        templatesDTW[label]
    except:
        templatesDTW[label]={}
    templatesDTW[label]=npy

#print(templatesDTW)
templateClasses = list(templatesDTW.keys())
#print(templateClasses)

ml_instances_paths=[]
for path in crossValidationDataPaths:
    #print(path)
    classFolders= os.listdir(path)
    for classFolder in classFolders:
        classFolder_path=f'{path}/{classFolder}'
        classTrials =os.listdir(classFolder_path)
        for trial in classTrials:
            trialPath =f'{classFolder_path}/{trial}'
            #print(trialPath)
            ml_instances_paths.append(trialPath)







for ml_instances_path in ml_instances_paths:
    label = ml_instances_path.split('/')
    classLabel = label[len(label)-1].split('_')[0]

    if MY_NODE_NO != int(classLabel):

        continue
    
    print('MY NODE NO is :',MY_NODE_NO," will generate DTW for class: ",classLabel)
    alreadyDTWFlag=False
    dir_path=dest_path
    i=0
    for i in range(3,len(label)-1):
        dir_path=dir_path+label[i]+'/'
    #print(dir_path)
    if os.path.exists(dir_path):
        pass       
        #print('exists')                  
                           
    else: 
        os.makedirs(dir_path)
        #print('created')   
    fileName =label[len(label)-1]
    filePath =dir_path+fileName
    if(os.path.exists(filePath)):
        print('already file exists:'+filePath)
        alreadyDTWFlag=True
    else:
        print('create file')
        alreadyDTWFlag=False

    if(alreadyDTWFlag ==True):
        continue

    npy =np.load(ml_instances_path)
    npy_matrix=npy.reshape(-1,NO_FEATURES)
    # print(npy_matrix.shape)
    #DTW CODE
    svm_feature_this_instance=[]
    for dtwclass_no in range(NO_CLASSES):
        dtwc=str(dtwclass_no)
        if dtwc not in templateClasses:
            print('template error')
            continue


        template_npy = templatesDTW[dtwc]
        template_matrix=template_npy.reshape(-1,NO_FEATURES)
        # print(template_matrix.shape)
        dtw_distances =np.zeros(NO_FEATURES)
        for col in range (NO_FEATURES):
            tempArray=template_matrix[:,col]
            compArray = npy_matrix[:,col]
            # print(tempArray.shape, compArray.shape)
            # print(tempArray)
            # print(compArray)

            #compDtw =retriveColumn(npy,col)
            #tempDtw=retriveColumn(template_npy,col)
            #later should change to windowed DTW
            dist,_=fastdtw(tempArray,compArray,dist=distance)
            
            
            dtw_distances[col]= dist # not using dtw_distances now
            #print('compared with:', dtwclass_no,' Feature# ',col)
        svm_feature_this_instance.append(dtw_distances)
        #print('compared with:', dtwclass_no)
    #now save to a file
    
    
    np.save(filePath,my_flatten(svm_feature_this_instance))
    

            
            
            
    




